In [38]:
import pandas as pd
import numpy as np

import csv

In [39]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

# import time

import time

In [40]:
# Set up WEBDRIVER

# Set up Chrome Options
chrome_options = Options()
chrome_options.add_argument("start-maximized")


driver = webdriver.Chrome(executable_path="chromedriver.exe", options=chrome_options)

# Anh có thể chỉnh tất cả filter rồi qua block tiếp theo

C:\Users\huybui1\AppData\Local\Temp\ipykernel_18084\3875490320.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="chromedriver.exe", options=chrome_options)


In [41]:
# prepare the csv file to store the data
open('data.csv', 'w').close()

# write header to the csv file
with open('data.csv', 'a', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['current_time', 'title','time_location','price','picture_links_list','item_link'])

# Go to each link and scrap data

## Core Scraping

In [42]:
def core_scraping(link):
    # open link
    driver.get(link)
    
    # wait until it's available and get all info

    # title
    title = WebDriverWait(driver, 5).\
        until(EC.presence_of_element_located((By.XPATH,'//div[@class="xyamay9 x1pi30zi x18d9i69 x1swvt13"]//h1'))).text
    print(title)

    # time and location
    time_location = driver.find_element(By.XPATH,'//span[@class="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x676frb x1nxh6w3 x1sibtaa xo1l8bm xi81zsa"]').text
    print(time_location)

    # price
    price = driver.find_element(By.XPATH,'//div[@class="x1xmf6yo"]//span[@class="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x676frb x1lkfr7t x1lbecb7 xk50ysn xzsf02u"]').text
    print(price)
    # get multiple picture links
    picture_links = driver.find_elements(By.XPATH, '//div[@class="x1a0syf3 x1ja2u2z"]//img')
    picture_links_list = []
    for i in range(len(picture_links)):  
        print(picture_links[i].get_attribute('src')) 
        picture_links_list.append(picture_links[i].get_attribute('src'))
    
    #Update time
    current_time = time.strftime('%Y-%m-%d %H:%M:%S')
    
    return current_time, title, time_location, picture_links_list, price


## Go over each link and go in them

In [46]:
def is_row_in_df(row):

    #Import
    # Try decoding the file using different encodings
    encodings = ['iso-8859-1', 'utf-8', 'cp1252']
    for encoding in encodings:
        try:
            df = pd.read_csv('data.csv', encoding=encoding)
            print("Successfully decoded file using encoding:", encoding)
            break
        except UnicodeDecodeError:
            print("Error decoding file using encoding:", encoding)
    
    #Check
    row = pd.DataFrame(row)
    is_in_df = row.isin(df)
    if is_in_df.any(axis=None):
        print("New listing")
        return True
    print("Old listing")
    return False

def loop_over_item(item_links):
    for item_link in item_links:
        # get all info
        try:
            current_time, title, time_location, picture_links_list, price = core_scraping(item_link)
        # print exception
        except Exception as e:
            print(e)
            print('Error at: ', item_link)
            # skip that item
            continue

        # create a row of data with dictionary
        data = {
            'current_time': current_time,
            'title': title,
            'time_location': time_location,
            'price': price,
            'picture_links_list': picture_links_list,
            'item_link': item_link
        }
        #Check if the data is in the excel file, if it is not, write data to csv file
        if is_row_in_df(data) == False:
            with open('data.csv', 'a', newline='') as f:
                dictWriter = csv.DictWriter(f, data.keys())
                dictWriter.writerow(data)



# Auto Scroll

In [47]:
def scroll(n_items):
    # Scroll down to load more items 10000 times
    for i in range(n_items):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)

    # Now loop through all items and get the info


# Scraping link to each item


In [48]:
# get all items
item = "Couch"

url = f"https://www.facebook.com/marketplace/tampa/search/?daysSinceListed=1&query={item}&exact=false"

# go to the url
driver.get(url)

# wait for the page to load
sleep(5)

# scroll down to load all items
scroll(5)

items = driver.find_elements(By.XPATH,'//a[@class="x1i10hfl xjbqb8w x6umtig x1b1mbwd xaqea5y xav7gou x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g x1lku1pv"]')

# list of all items' link
item_links = []

print(len(items))
# loop through all items
for item in items:
    print(item.get_attribute('href'))
    item_links.append(item.get_attribute('href'))

# go into each item and get the info
loop_over_item(item_links)

140
https://www.facebook.com/marketplace/item/720792272823643/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
https://www.facebook.com/marketplace/item/562980342474364/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
https://www.facebook.com/marketplace/item/224547316598058/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
https://www.facebook.com/marketplace/item/719106413201405/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
https://www.facebook.com/marketplace/item/3486367308352913/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
https://www.facebook.com/marketplace/item/231607999302137/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
https://www.facebook.com/marketplace/item/1230095521201952/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
https://www.facebook.com/marketplace/item/226012199789059/?ref=search&referral_code=null&referral_story_t

https://www.facebook.com/marketplace/item/151535611105731/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
https://www.facebook.com/marketplace/item/193052843419983/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
https://www.facebook.com/marketplace/item/745247947201150/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
https://www.facebook.com/marketplace/item/1329633750945684/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
https://www.facebook.com/marketplace/item/669253804950678/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
https://www.facebook.com/marketplace/item/1228498461086043/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
https://www.facebook.com/marketplace/item/1218833888768621/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
https://www.facebook.com/marketplace/item/542712867839436/?ref=search&referral_code=null&referral_story_type

Couch
Listed 22 minutes ago in St Petersburg, FL
$90
https://scontent-hou1-1.xx.fbcdn.net/v/t45.5328-4/330914553_5943398439110008_9085970278289227725_n.jpg?stp=dst-jpg_s960x960&_nc_cat=109&ccb=1-7&_nc_sid=c48759&_nc_ohc=plO0JG48XEkAX9ZgS1L&_nc_ht=scontent-hou1-1.xx&oh=00_AfCP9azlv9r3vKrwKE02naNPXUU548JLJi6SHBkO7srH0g&oe=64068192
https://scontent-hou1-1.xx.fbcdn.net/v/t45.5328-4/330069087_5860445230736038_1059336534341228784_n.jpg?stp=dst-jpg_s960x960&_nc_cat=108&ccb=1-7&_nc_sid=c48759&_nc_ohc=87Nw28lWPp8AX-fuEq1&_nc_ht=scontent-hou1-1.xx&oh=00_AfCzltPJbTNGAh_C1wP8GiuKoD9MeY6USO4ftW9RPnkacg&oe=64058FEA
https://scontent-hou1-1.xx.fbcdn.net/v/t45.5328-4/329642675_5739167996209200_5890971102330263446_n.jpg?stp=dst-jpg_s960x960&_nc_cat=102&ccb=1-7&_nc_sid=c48759&_nc_ohc=UroHNlMNtRQAX92OArb&_nc_ht=scontent-hou1-1.xx&oh=00_AfD_soA2i6O7acCPXH_41LSe3Iy60NDWnTlYAMvZxjtsng&oe=64064ECA
Error decoding file using encoding: utf-8
Successfully decoded file using encoding: iso-8859-1
Sofa Chaise Couch 

Couch And Loveseat
Listed 5 hours ago in Tampa, FL
$225
https://scontent-hou1-1.xx.fbcdn.net/v/t45.5328-4/332380926_8784055501666902_7202757262523465132_n.jpg?stp=dst-jpg_p720x720&_nc_cat=110&ccb=1-7&_nc_sid=c48759&_nc_ohc=OCrl4kv4ZtMAX-4SI-q&_nc_ht=scontent-hou1-1.xx&oh=00_AfAKGujpypogx19RufvEVkHkvXocXhxe2AWk13W9LK8sYg&oe=640524F3
https://scontent-hou1-1.xx.fbcdn.net/v/t45.5328-4/330501989_8773144796093071_3853337307452030949_n.jpg?stp=dst-jpg_p720x720&_nc_cat=111&ccb=1-7&_nc_sid=c48759&_nc_ohc=kN2CO-jqlbEAX8UGbOL&_nc_ht=scontent-hou1-1.xx&oh=00_AfDQGo-7EPxp4Wr6iPNsw8Hckfpl1nO3HA2bJ5vnf0swMQ&oe=6405F721
Error decoding file using encoding: utf-8
Successfully decoded file using encoding: iso-8859-1
Couch
Listed 28 minutes ago in Pinellas Park, FL
$150
https://scontent-hou1-1.xx.fbcdn.net/v/t45.5328-4/323544457_6177084618979393_8228339949966146525_n.jpg?stp=dst-jpg_p720x720&_nc_cat=106&ccb=1-7&_nc_sid=c48759&_nc_ohc=6RWUcxPOmYEAX9IDgU0&_nc_ht=scontent-hou1-1.xx&oh=00_AfBc1HtrB8wtcCZWMR-6

Couch
Listed 17 hours ago in Tampa, FL
$1,200
https://scontent-hou1-1.xx.fbcdn.net/v/t45.5328-4/333062048_5691551537623412_5783998294558150831_n.jpg?stp=dst-jpg_s960x960&_nc_cat=100&ccb=1-7&_nc_sid=c48759&_nc_ohc=q16WCt5NlygAX_m3Cs5&_nc_ht=scontent-hou1-1.xx&oh=00_AfDn1SMzj7iyqI-Oe8f6oO91QP4OiUKYclgWEByOmBeXxw&oe=6404ED5E
https://scontent-hou1-1.xx.fbcdn.net/v/t45.5328-4/333583535_5405891582846280_284514806806845347_n.jpg?stp=dst-jpg_s960x960&_nc_cat=106&ccb=1-7&_nc_sid=c48759&_nc_ohc=UHib9ZF-6bAAX_xoLgZ&_nc_ht=scontent-hou1-1.xx&oh=00_AfCVUDhdbBiHRDrMZ8-_0DuWeBnKip_-KUnatB12_RD3Fg&oe=6406B19D
https://scontent-hou1-1.xx.fbcdn.net/v/t45.5328-4/333749323_5781172792008477_4298255735755580541_n.jpg?stp=dst-jpg_s960x960&_nc_cat=101&ccb=1-7&_nc_sid=c48759&_nc_ohc=WNJUHca8MCMAX80ex2S&_nc_ht=scontent-hou1-1.xx&oh=00_AfBHrtaItu1ZOic7MjGBg7ZqHbdV72Yf859kunTej3xkmw&oe=6405FB78
https://scontent-hou1-1.xx.fbcdn.net/v/t45.5328-4/327463197_4331409633649894_4230061653945386126_n.jpg?stp=dst-jpg_s960x9

Storage Sectional
Listed a day ago in Tampa, FL
$400
https://scontent-hou1-1.xx.fbcdn.net/v/t45.5328-4/333726151_6907838622583230_1112017883978268545_n.jpg?stp=dst-jpg_s960x960&_nc_cat=106&ccb=1-7&_nc_sid=c48759&_nc_ohc=YEpylz1a950AX9eBIpN&_nc_ht=scontent-hou1-1.xx&oh=00_AfBSv08oK3nOLA1-7qbUxsgS7LKJ3ch3tTKXY-hG3n8UHA&oe=6405DF7F
https://scontent-hou1-1.xx.fbcdn.net/v/t45.5328-4/331226746_6208173345892779_3459182756905091408_n.jpg?stp=dst-jpg_s960x960&_nc_cat=104&ccb=1-7&_nc_sid=c48759&_nc_ohc=dpwm-GUWMNkAX8UTbze&_nc_ht=scontent-hou1-1.xx&oh=00_AfBDv4aqg2xsSqUAo6fCZNgUKfxpTdTzZWnEhNId-o7nfg&oe=640648B7
https://scontent-hou1-1.xx.fbcdn.net/v/t45.5328-4/332582171_5465168403583740_1889448239245247913_n.jpg?stp=dst-jpg_s960x960&_nc_cat=107&ccb=1-7&_nc_sid=c48759&_nc_ohc=pgOe1iY44h4AX9ojama&_nc_ht=scontent-hou1-1.xx&oh=00_AfC4gQn0f-7btxv2khrOe-PYvf980XGZzc0fy3nYXiWbdQ&oe=6404DFFF
https://scontent-hou1-1.xx.fbcdn.net/v/t45.5328-4/322931159_5946702985415242_4107795728176463188_n.jpg?stp=dst-jp

Sectional
Listed a day ago in Tampa, FL
$400
https://scontent-hou1-1.xx.fbcdn.net/v/t45.5328-4/332264684_6082380295151661_8357802129538915021_n.jpg?stp=dst-jpg_p180x540&_nc_cat=109&ccb=1-7&_nc_sid=c48759&_nc_ohc=No2ZJ61LRVwAX_gKh0Q&_nc_ht=scontent-hou1-1.xx&oh=00_AfB0qEX5wdoM5WR_MmV46hSuHOQ1CCR82uHjSDR9OdqsQg&oe=64053BDD
https://scontent-hou1-1.xx.fbcdn.net/v/t45.5328-4/315210574_6285900668088174_5197964571751936480_n.jpg?stp=dst-jpg_p180x540&_nc_cat=105&ccb=1-7&_nc_sid=c48759&_nc_ohc=1Mj90bFfWgsAX8iX03J&_nc_ht=scontent-hou1-1.xx&oh=00_AfDQY3-F87n_XRqafCLreC5IZZ9wrUPZkeXR8zZZ1vXXrw&oe=6404C75F
https://scontent-hou1-1.xx.fbcdn.net/v/t45.5328-4/329273185_5884025548301699_117303569026550578_n.jpg?stp=dst-jpg_p180x540&_nc_cat=109&ccb=1-7&_nc_sid=c48759&_nc_ohc=TlvZNOzIEk4AX_TqRll&_nc_ht=scontent-hou1-1.xx&oh=00_AfCvF3Sy2zQtpvYBkjJvWMPZilIlt52lZs3dTMS5P39TdQ&oe=640593F6
https://scontent-hou1-1.xx.fbcdn.net/v/t45.5328-4/327777847_5592267247545414_6036505564454684385_n.jpg?stp=dst-jpg_p180x54

Teal Blue couch in LIKE NEW CONDITION
Listed 17 hours ago in New Port Richey, FL
$300
https://scontent-hou1-1.xx.fbcdn.net/v/t45.5328-4/333697413_5885373691539930_8639365979323424403_n.jpg?stp=dst-jpg_s960x960&_nc_cat=102&ccb=1-7&_nc_sid=c48759&_nc_ohc=ED2xKpWKvLwAX-FxvWG&_nc_ht=scontent-hou1-1.xx&oh=00_AfARlVYQimSOFLZq0INTgNl5smwzvk2DiZIgZHOxRVus4A&oe=64062457
https://scontent-hou1-1.xx.fbcdn.net/v/t45.5328-4/332952971_8833362730038834_5325352755134568346_n.jpg?stp=dst-jpg_s960x960&_nc_cat=103&ccb=1-7&_nc_sid=c48759&_nc_ohc=jYDbpKdFvyYAX_upl-u&_nc_ht=scontent-hou1-1.xx&oh=00_AfDgixdxgGI1x6dfbym-OEP5_bifijH9PCQ8mbplqb7tVw&oe=6404F710
https://scontent-hou1-1.xx.fbcdn.net/v/t45.5328-4/333820586_9561402833885321_2681900094831256973_n.jpg?stp=dst-jpg_s960x960&_nc_cat=109&ccb=1-7&_nc_sid=c48759&_nc_ohc=tBqLVd8_LMEAX_CbNjJ&_nc_ht=scontent-hou1-1.xx&oh=00_AfArb1OJJf9agPX2XxwqNAkS1xVBPpmfOGNMHJY9TRfEdQ&oe=64055E18
Error decoding file using encoding: utf-8
Successfully decoded file using encodi

KeyboardInterrupt: 

In [129]:
driver.quit()